In [107]:
import pygame
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.lines import Line2D
import matplotlib.patches as patches
import torch
import torch.nn as nn
import torch.nn.functional as F
from scipy import ndimage


class MinimalCNN(nn.Module):
    def __init__(self):
        super(MinimalCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 4, kernel_size=5, stride=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(4, 8, kernel_size=3, stride = 1)
        self.conv3 = nn.Conv2d(8, 10, kernel_size=4, stride = 1)
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        conv1 = self.conv1(x)
        pool1 = self.pool(F.relu(conv1))
        conv2 = self.conv2(pool1)
        pool2 = self.pool(F.relu(conv2))
        conv3 = self.conv3(pool2)
        conv3 = self.dropout(conv3)
        sum = torch.sum(conv3, dim=(2, 3))
        out = sum.view(-1, 10)
        return conv1, pool1, conv2, pool2, conv3, sum, out
        

def load_model():
    model = MinimalCNN()
    model.load_state_dict(torch.load('mnist_cnn_4-8-10.pth', map_location='cpu'))
    model.eval()
    return model
    

def center_and_normalize_digit(digit_array, target_size=20):
    rows, cols = np.where(digit_array > 0)
    
    if len(rows) == 0:
        return digit_array
    
    min_row, max_row = rows.min(), rows.max()
    min_col, max_col = cols.min(), cols.max()
    
    digit_region = digit_array[min_row:max_row+1, min_col:max_col+1]
    
    height, width = digit_region.shape
    scale = min(target_size / height, target_size / width)
    
    if scale < 1.0:
        new_height = int(height * scale)
        new_width = int(width * scale)
        digit_region = ndimage.zoom(digit_region, (new_height/height, new_width/width))
    
    centered_digit = np.zeros((28, 28))
    
    region_height, region_width = digit_region.shape
    start_row = (28 - region_height) // 2
    start_col = (28 - region_width) // 2
    
    centered_digit[start_row:start_row+region_height, start_col:start_col+region_width] = digit_region
    
    return centered_digit
    

def calculate_center_of_mass(digit_array):
    rows, cols = np.where(digit_array > 0)
    if len(rows) == 0:
        return 14, 14
    
    weights = digit_array[rows, cols]
    center_row = np.average(rows, weights=weights)
    center_col = np.average(cols, weights=weights)
    
    return center_row, center_col
    

def fine_tune_centering(digit_array):
    center_row, center_col = calculate_center_of_mass(digit_array)
    
    target_center = 13.5
    shift_row = int(round(target_center - center_row))
    shift_col = int(round(target_center - center_col))
    
    shift_row = max(-10, min(10, shift_row))
    shift_col = max(-10, min(10, shift_col))
    
    if shift_row != 0 or shift_col != 0:
        digit_array = ndimage.shift(digit_array, (shift_row, shift_col), cval=0.0)
    
    return digit_array
    

def preprocess_drawing(digit_array):
    centered = center_and_normalize_digit(digit_array)
    final = fine_tune_centering(centered)
    
    tensor = torch.FloatTensor(final)
    normalized = (tensor - 0.5) / 0.5
    input_tensor = normalized.unsqueeze(0).unsqueeze(0)
    
    return input_tensor, final
    

def predict_digit_improved(model, digit_array):
    input_tensor, processed_array = preprocess_drawing(digit_array)
    stages = []
    with torch.no_grad():
        conv1, pool1, conv2, pool2, conv3, sum, outputs = model(input_tensor)
        stages.append(conv1)
        stages.append(pool1)
        stages.append(conv2)
        stages.append(pool2)
        stages.append(conv3)
        stages.append(sum)
        
        probabilities = F.softmax(outputs, dim=1)
        confidence, predicted = torch.max(probabilities, 1)
    
    return predicted.item(), confidence.item(), probabilities.squeeze().numpy(), processed_array, stages
    

def draw_digit():
    WIDTH, HEIGHT = 560, 560
    BG_COLOR = (255, 255, 255)
    
    running = True
    grid_array = []
    
    pygame.init()
    window = pygame.display.set_mode((WIDTH, HEIGHT))
    pygame.display.set_caption("Draw Digit - Press ENTER when done")
    window.fill(BG_COLOR)
    
    def draw_grid():
        block_size = int(WIDTH/28)
        
        for i in range(28):
            for j in range(28):
                grid_block = pygame.Rect(i * block_size, j * block_size, block_size, block_size)
                color = (255, 255, 255)
                grid_array.append((grid_block, color))
    
    def draw_rectangles():
        for rect, color in grid_array:
            pygame.draw.rect(window, color, rect)
            pygame.draw.rect(window, (200, 200, 200), rect, 1)
    
    def get_digit_array():
        digit = np.zeros((28, 28))
        
        for row in range(28):
            for col in range(28):
                index = col * 28 + row
                rect, color = grid_array[index]
                
                if color == (0, 0, 0):
                    digit[row, col] = 0.9
        
        return digit
    
    def clear_grid():
        for i in range(len(grid_array)):
            rect, _ = grid_array[i]
            grid_array[i] = (rect, (255, 255, 255))
    
    draw_grid()
    
    print("- Click and drag to draw")
    print("- Press ENTER when finished")
    print("- Press SPACE to clear")
    
    digit_array = None
    clock = pygame.time.Clock()
    
    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
                
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_RETURN:
                    digit_array = get_digit_array()
                    running = False
                    
                elif event.key == pygame.K_SPACE:
                    clear_grid()
        
        if pygame.mouse.get_pressed()[0]:
            mouse_pos = pygame.mouse.get_pos()
            for index, (rect, color) in enumerate(grid_array):
                if rect.collidepoint(mouse_pos):
                    grid_array[index] = (rect, (0, 0, 0))
        
        window.fill(BG_COLOR)
        draw_rectangles()
        pygame.display.flip()
        clock.tick(60)
    
    pygame.quit()
    return digit_array
    

class CNNVisualizer:
    def __init__(self, input_size=(28, 28), filter_size1=(5, 5), filter_size2 = (3, 3), filter_size3 = (4, 4), num_filters1 = 4, num_channels1 = 1, num_filters2 = 8, num_channels2 = 4, num_filters3 = 10, num_channels3 = 8):
        self.input_size = input_size
        self.filter_size1 = filter_size1
        self.filter_size2 = filter_size2
        self.filter_size3 = filter_size3         
        self.num_filters1 = num_filters1
        self.num_filters2 = num_filters2
        self.num_filters3 = num_filters3
        self.num_channels1 = num_channels1
        self.num_channels2 = num_channels2
        self.num_channels3 = num_channels3
        self.conv1_size = (
            input_size[0] - filter_size1[0] + 1,
            input_size[1] - filter_size1[1] + 1
        )
        self.conv2_size = (
            self.conv1_size[0]/2 - filter_size2[0] + 1,
            self.conv1_size[1]/2 - filter_size2[1] + 1
        )
        self.conv3_size = (
            self.conv2_size[0]/2 - filter_size3[0] + 1,
            self.conv2_size[1]/2 - filter_size3[1] + 1
        )
        
        # Load model and get prediction
        self.model = load_model()
        self.input_data = draw_digit()
        
        # Get prediction
        if self.input_data is not None and np.sum(self.input_data) > 0:
            self.predicted_digit, self.confidence, _, _, stages = predict_digit_improved(self.model, self.input_data)
        else:
            self.predicted_digit, self.confidence = None, None
                
        filters1 = self.model.conv1.weight.data
        filters2 = self.model.conv2.weight.data
        filters3 = self.model.conv3.weight.data
        filter_bias1 = self.model.conv1.bias.data
        filter_bias2 = self.model.conv2.bias.data
        filter_bias3 = self.model.conv3.bias.data
        self.filters1 = []
        self.filters2 = []
        self.filters3 = []
        self.filter_bias1 = filter_bias1.detach().cpu().numpy()
        self.filter_bias2 = filter_bias2.detach().cpu().numpy()
        self.filter_bias3 = filter_bias3.detach().cpu().numpy()
        for i in range(filters1.size(0)):
            channels = []
            for j in range(num_channels1):
                channels.append(filters1[i][j].detach().cpu().numpy())
            self.filters1.append(channels)

        
        for i in range(filters2.size(0)):
            channels = []
            for j in range(num_channels2):
                channels.append(filters2[i][j].detach().cpu().numpy())
            self.filters2.append(channels)

        for i in range(filters3.size(0)):
            channels = []
            for j in range(num_channels3):
                channels.append(filters3[i][j].detach().cpu().numpy())
            self.filters3.append(channels)

        # Generate feature maps
        if self.input_data is not None:
            self.output_maps1 = stages[0].squeeze(0).detach().cpu().numpy()
            self.output_maps2 = stages[2].squeeze(0).detach().cpu().numpy()
            self.channels2 = self.apply_convolution(num_filters2, num_channels2, filters2, stages[1])
            self.output_maps3 = stages[4].squeeze(0).detach().cpu().numpy()
            # self.channels3 = self.apply_convolution(num_filters3, num_channels3, filters3, stages[3])
            self.pooling1 = stages[1].squeeze(0).detach().cpu().numpy()
            self.pooling2 = stages[3].squeeze(0).detach().cpu().numpy()
            self.out = stages[5].squeeze(0).detach().cpu().numpy()
        else:
            self.output_maps1 = []
            self.output_maps2 = []
            self.output_maps3 = []
            self.pooling1 = []
            self.pooling2 = []
            

    
    def apply_convolution(self, num_kernels, num_channels, weights, input):
        output_maps = []
        for i in range(0, num_kernels):
            kernels = []
            for j in range(0, num_channels):
                conv_output = F.conv2d(input[:, j:j+1, :, :], weights[i:i+1, j:j+1, :, :], stride = 1)
                conv_output_np = conv_output.squeeze(0).squeeze(0).detach().cpu().numpy()
                kernels.append(conv_output_np)
            output_maps.append(kernels)
        return output_maps

    
    def visualize_full_matrices(self, show_numbers_on_filters=True, decimals=1):
        if self.input_data is None:
            print("No drawing to visualize!")
            return

        # Set up the figure with a larger size
        fig = plt.figure(figsize=(40, 30))  # Increased from (30, 20)

        def plot_matrix(ax, matrix, title, cmap='Blues', value_format='.1f', show_grid=True, show_numbers=False, change_color = True, font = -1):
            im = ax.imshow(matrix, cmap=cmap, interpolation='nearest')
            ax.set_title(title, fontsize=50, fontweight='bold') 

            if show_grid and matrix.shape[0] <= 10 and matrix.shape[1] <= 10:
                ax.set_xticks(np.arange(-.5, matrix.shape[1], 1), minor=True)
                ax.set_yticks(np.arange(-.5, matrix.shape[0], 1), minor=True)
                ax.grid(which='minor', color='w', linestyle='-', linewidth=2)
                ax.set_xticks([])
                ax.set_yticks([])
            else:
                ax.set_xticks([])
                ax.set_yticks([])

            if show_numbers and matrix.shape[0] <= 10 and matrix.shape[1] <= 10:
                max_dim = max(matrix.shape[0], matrix.shape[1])
                if font > 0:
                    fontsize = font
                elif matrix.shape[0] == 1 and matrix.shape[1] == 1:
                    fontsize = 60  # Smaller font size for 1x1 matrices
                elif max_dim <= 3:
                    fontsize = 44  # Larger font size for small matrices
                elif max_dim <= 5:
                    fontsize = 35  # Medium font size for medium matrices
                else:
                    fontsize = 20  # Smaller font size for larger matrices
                for i in range(matrix.shape[0]):
                    for j in range(matrix.shape[1]):
                        value = matrix[i, j]
                        text_color = 'white' if (abs(value) > 0.3 and change_color) else 'black'
                        if isinstance(value, (int, np.integer)):
                            value_str = f"{value}"
                        else:
                            value_str = f"{value:{value_format}}"
                        ax.text(j, i, value_str, ha='center', va='center',
                                color=text_color, fontsize=fontsize)
            return ax

        # Adjusted left positions to accommodate larger subplots
        input_left = 0.01
        kernel1_base_left = 0.35
        feature1_base_left = 0.55
        pooling1_base_left = 0.75
        kernels2_base_left = 1
        channels2_base_left = 1.4
        feature2_base_left = 1.7
        pooling2_base_left = 1.9
        feature3_base_left = 2.1
        out_base_left = 2.3
        prediction_left = 2.5

        # Calculate true center position
        sub_height = 0.12  # Increased from 0.06
        v_spacing = 0.02  # Increased from 0.0001
        top_edge = 0.85 + sub_height
        bottom_edge = (0.85 - (3 * 0.3)) - (sub_height + v_spacing)  # Adjusted for larger blocks
        true_center = (top_edge + bottom_edge) / 2

        # Center input and prediction boxes
        box_height = 0.25  # Increased from 0.15
        centered_bottom = true_center - box_height / 2

        # Plot the input data
        input_width = 0.25  # Increased from 0.15
        ax_input = fig.add_axes([input_left, centered_bottom, input_width, box_height])
        plot_matrix(ax_input, self.input_data, 'Input Drawing\n(28×28)', show_numbers=False)
        ax_input.set_title('Input Drawing\n(28×28)', fontsize=80, fontweight='bold')

        # Store axes for arrows
        kernel1_block_axes = []
        output1_block_axes = []
        pooling1_block_axes = []

        num_kernels = 4
        total_height = num_kernels * sub_height + (num_kernels - 1) * v_spacing

        # 1st column: Kernels (K1-K4)
        for row in range(4):
            left_kernel = kernel1_base_left
            left_feature = feature1_base_left
            left_pooling = pooling1_base_left
            bottom = 0.77 - (row * 0.3)  # Increased spacing between rows

            # Plot kernel
            ax_filter = fig.add_axes([left_kernel, bottom, 0.14, 0.14])  # Increased width from 0.06
            plot_matrix(ax_filter, self.filters1[row][0], f'K1_{row+1}', cmap='RdBu', show_numbers=show_numbers_on_filters)
            kernel1_block_axes.append((row, ax_filter))

            # Plot corresponding feature map
            ax_output = fig.add_axes([left_feature, bottom, 0.14, 0.14])
            plot_matrix(ax_output, self.output_maps1[row], f'FM1_{row+1}', show_numbers=False)
            output1_block_axes.append((row, ax_output))

            # Plot pooled feature maps
            ax_pooling = fig.add_axes([left_pooling, bottom, 0.14, 0.14])
            plot_matrix(ax_pooling, self.pooling1[row], f'P1_{row+1}', cmap='Blues', show_numbers=False)
            pooling1_block_axes.append((row, ax_pooling))
        

        def add_straight_arrow(fig, start_x, start_y, end_x, end_y, color='purple', width=3.0):  # Increased width
            arrow = patches.FancyArrowPatch(
            (start_x, start_y), (end_x, end_y),
            arrowstyle='->', color=color, linewidth=width,
            connectionstyle='arc3,rad=0',
            shrinkA=3, shrinkB=3,
            mutation_scale=80,  # Increased for much larger arrowhead
            transform=fig.transFigure, zorder=0
            )
            fig.add_artist(arrow)

        def add_straight_line(fig, start_x, start_y, end_x, end_y, color='purple', width=3.0):
            from matplotlib.lines import Line2D
            line = Line2D(
                [start_x, end_x], [start_y, end_y],
                color=color, linewidth=width,
                transform=fig.transFigure, zorder=0
            )
            fig.add_artist(line)

        kernel2_block_axes = []
        channels2_block_axes = []

        # Plot kernels and feature maps in 4 vertical 2x2 blocks
        for block_num in range(8):
            block_kernel_axes = []
            block_output_axes = []
            block_top = 1.45 - (block_num * 0.3)  # Increased spacing

            for sub_row in range(2):
                for sub_col in range(2):
                    f = sub_row * 2 + sub_col
                    sub_width = 0.10  # Increased from 0.05
                    sub_height = 0.10  # Increased from 0.05
                    h_spacing_kernels = 0.12  # Increased from 0.07
                    h_spacing_features = 0.03  # Increased from 0.015
                    v_spacing = 0.03  # Increased from 0.015

                    left_kernel = kernels2_base_left + sub_col * (sub_width + h_spacing_kernels)
                    left_feature = channels2_base_left + sub_col * (sub_width + h_spacing_features)
                    bottom = block_top - sub_row * (sub_height + v_spacing)

                    # Plot kernel
                    ax_filter = fig.add_axes([left_kernel, bottom, sub_width, sub_height])
                    plot_matrix(ax_filter, self.filters2[block_num][f], f'K{block_num+1}C{f+1}', cmap='RdBu', show_numbers=show_numbers_on_filters)
                    block_kernel_axes.append(ax_filter)

                    line_y = bottom + sub_height / 2
                    line_x_start = left_kernel - 0.05  # Adjusted for larger layout
                    line_x_end = left_kernel
                    add_straight_line(fig, line_x_start, line_y, line_x_end, line_y, color='purple', width=3.0)

                    # Add text label
                    text_x = line_x_start - 0.02
                    text_y = line_y
                    fig.text(text_x, text_y, f'$P1_{{{f+1}}}$', fontsize=48, verticalalignment='center', horizontalalignment='right')

                    # Plot corresponding feature map
                    ax_output = fig.add_axes([left_feature, bottom, sub_width, sub_height])
                    plot_matrix(ax_output, self.channels2[block_num][f], f'FM{block_num+1}C{f+1}', show_numbers=False)
                    block_output_axes.append(ax_output)

            kernel2_block_axes.append((block_num, block_kernel_axes))
            channels2_block_axes.append((block_num, block_output_axes))

        output2_block_axes = []
        pooling2_block_axes = []

        # Plot feature maps and pooling in 1 column of 8
        for row in range(8):
            bottom = 1.375 - (row * 0.3)  # Increased spacing
            sub_height = 0.12  # Increased from 0.06
            left_feature = feature2_base_left
            left_pooling = pooling2_base_left

            # Plot feature map
            ax_output = fig.add_axes([left_feature, bottom, 0.12, sub_height])
            plot_matrix(ax_output, self.output_maps2[row], f'FM2_{row+1}', show_numbers=False)
            output2_block_axes.append((row, ax_output))

            # Plot pooled feature map
            ax_pooling = fig.add_axes([left_pooling, bottom, 0.12, sub_height])
            plot_matrix(ax_pooling, self.pooling2[row], f'P2_{row+1}', cmap='Blues', show_numbers=False)
            pooling2_block_axes.append((row, ax_pooling))

        feature3_block_axes = []
        output_block_axes = []
        for row in range(10):
            bottom = 1.23 - (row * 0.2)  # Increased spacing
            sub_height = 0.12  # Increased from 0.06
            left_feature = feature3_base_left

            # Plot conv3 output
            ax_feature = fig.add_axes([left_feature, bottom, 0.12, sub_height])
            plot_matrix(ax_feature, self.output_maps3[row], f'FM3_{row+1}')
            feature3_block_axes.append((row, ax_feature))

            # Plot output
            ax_out = fig.add_axes([out_base_left, bottom, 0.12, sub_height])
            out_value = np.array([[self.out[row]]])
            plot_matrix(ax_out, out_value, f'SUM_{row}', show_numbers=True, value_format = '.3f', cmap='Blues', change_color = False)
            output_block_axes.append((row, ax_out))

        if self.predicted_digit is not None:
            prediction_width = 0.25  # Increased from 0.15
            prediction_height = 0.25  # Increased from 0.15
            prediction_bottom = centered_bottom
            ax_prediction = fig.add_axes([prediction_left, prediction_bottom, prediction_width, prediction_height])

            # Create a square with the predicted number
            ax_prediction.text(0.5, 0.5, str(self.predicted_digit), ha='center', va='center',
                            fontsize=250, fontweight='bold', transform=ax_prediction.transAxes)

            ax_prediction.set_xlim(0, 1)
            ax_prediction.set_ylim(0, 1)
            ax_prediction.set_xticks([])
            ax_prediction.set_yticks([])
            ax_prediction.set_title('CNN Prediction', fontsize=60, fontweight='bold')

            # Add border
            for spine in ax_prediction.spines.values():
                spine.set_visible(True)
                spine.set_linewidth(2)  # Increased linewidth

            # Add confidence text
            confidence_percentage = self.confidence * 100
            fig.text(prediction_left + prediction_width/2, prediction_bottom - 0.05,
                    f'Confidence: {confidence_percentage:.1f}%',
                    ha='center', va='top', fontsize=60, transform=fig.transFigure)
       
       #Add lines from input to kernels
        for row, ax_filter in kernel1_block_axes:
            line_y_start = ax_input.get_position().y0 + ax_input.get_position().height / 2
            line_y_end = ax_filter.get_position().y0 + ax_filter.get_position().height / 2
            line_x_start = input_left + input_width
            line_x_end = ax_filter.get_position().x0
            add_straight_line(fig, line_x_start, line_y_start, line_x_end, line_y_end, color='purple', width=3.0)

        # Add arrows from each kernel to its corresponding feature map
        for row, (ax_filter) in kernel1_block_axes:
            _, ax_output = output1_block_axes[row]
            line_y_start = ax_filter.get_position().y0 + ax_filter.get_position().height / 2
            line_y_end = ax_output.get_position().y0 + ax_output.get_position().height / 2
            line_x_start = ax_filter.get_position().x0 + ax_filter.get_position().width
            line_x_end = ax_output.get_position().x0
            add_straight_arrow(fig, line_x_start, line_y_start, line_x_end, line_y_end, color='purple', width=3.0)
        
        # Plot conv1 bias values on top of the arrow between each kernel and its corresponding feature map
        for row, (ax_filter) in kernel1_block_axes:
            _, ax_output = output1_block_axes[row]
            line_y_start = ax_filter.get_position().y0 + ax_filter.get_position().height / 2
            line_y_end = ax_output.get_position().y0 + ax_output.get_position().height / 2
            line_x_start = ax_filter.get_position().x0 + ax_filter.get_position().width
            line_x_end = ax_output.get_position().x0
            
            # Add the bias value as text on the arrow
            bias_value = self.filter_bias1[row]
            # Plot the bias value as a 1x1 matrix using plot_matrix
            bias_matrix = np.array([[bias_value]])
            # Calculate the position for the bias matrix: center between the two axes
            bias_width = 0.06
            bias_height = 0.05
            bias_center_x = ((line_x_start + line_x_end) / 2) - bias_width / 2
            bias_center_y = ((line_y_start + line_y_end) / 2) - bias_height / 2
            ax_bias = fig.add_axes([bias_center_x, bias_center_y, bias_width, bias_height])
            plot_matrix(ax_bias, bias_matrix, '', show_numbers=True, value_format='.2f', show_grid=False, change_color= False, font = 40)

        # Add arrows from each feature map to its corresponding pooling map
        for row, (ax_output) in output1_block_axes:
            _, ax_pooling = pooling1_block_axes[row]
            line_y_start = ax_output.get_position().y0 + ax_output.get_position().height / 2
            line_y_end = ax_pooling.get_position().y0 + ax_pooling.get_position().height / 2
            line_x_start = ax_output.get_position().x0 + ax_output.get_position().width
            line_x_end = ax_pooling.get_position().x0
            add_straight_arrow(fig, line_x_start, line_y_start, line_x_end, line_y_end, color='purple', width=3.0)
        
        for block_num, block_kernel_axes in kernel2_block_axes:
            # Get the right center of the 2x2 kernel block
            ax_k0 = block_kernel_axes[0]
            ax_k3 = block_kernel_axes[3]
            kernel_block_x = ax_k3.get_position().x0 + ax_k3.get_position().width + 0.01
            kernel_block_y = (ax_k0.get_position().y0 + ax_k3.get_position().y0 + ax_k3.get_position().height) / 2

            # Get the left center of the 2x2 feature (channels) block
            block_output_axes = channels2_block_axes[block_num][1]
            ax_f0 = block_output_axes[0]
            ax_f3 = block_output_axes[3]
            filter_block_x = ax_f0.get_position().x0 - 0.01
            filter_block_y = (ax_f0.get_position().y0 + ax_f3.get_position().y0 + ax_f3.get_position().height) / 2

            add_straight_arrow(fig, kernel_block_x, kernel_block_y, filter_block_x, filter_block_y, color='purple', width=3.0)
        
        # Add arrows from each 2x2 channel block (channels2) to its corresponding feature map (output2)
        for block_num, block_output_axes in channels2_block_axes:
            # Get the right center of the 2x2 channel block
            ax_c0 = block_output_axes[0]
            ax_c3 = block_output_axes[3]
            channel_block_x = ax_c3.get_position().x0 + ax_c3.get_position().width + 0.01
            channel_block_y = (ax_c0.get_position().y0 + ax_c3.get_position().y0 + ax_c3.get_position().height) / 2

            # Get the left center of the corresponding feature map in output2_block_axes
            _, ax_feature = output2_block_axes[block_num]
            feature_block_x = ax_feature.get_position().x0 - 0.01
            feature_block_y = ax_feature.get_position().y0 + ax_feature.get_position().height / 2

            add_straight_arrow(fig, channel_block_x, channel_block_y, feature_block_x, feature_block_y, color='purple', width=3.0)

        #Plot conv2 bias values on top of the arrow between each channel and its corresponding feature map
        for block_num, block_output_axes in channels2_block_axes:
            # Get the right center of the 2x2 channel block
            ax_c0 = block_output_axes[0]
            ax_c3 = block_output_axes[3]
            channel_block_x = ax_c3.get_position().x0 + ax_c3.get_position().width + 0.01
            channel_block_y = (ax_c0.get_position().y0 + ax_c3.get_position().y0 + ax_c3.get_position().height) / 2

            # Get the left center of the corresponding feature map in output2_block_axes
            _, ax_feature = output2_block_axes[block_num]
            feature_block_x = ax_feature.get_position().x0 - 0.01
            feature_block_y = ax_feature.get_position().y0 + ax_feature.get_position().height / 2
            
            # Add the bias value as text on the arrow
            bias_value = self.filter_bias2[block_num]
            # Plot the bias value as a 1x1 matrix using plot_matrix
            bias_matrix = np.array([[bias_value]])
            # Calculate the position for the bias matrix: center between the two axes
            bias_width = 0.06
            bias_height = 0.05
            bias_center_x = ((channel_block_x + feature_block_x) / 2) - bias_width / 2
            bias_center_y = ((channel_block_y + feature_block_y) / 2) - bias_height / 2
            ax_bias = fig.add_axes([bias_center_x, bias_center_y, bias_width, bias_height])
            plot_matrix(ax_bias, bias_matrix, '', show_numbers=True, value_format='.2f', show_grid=False, change_color= False, font = 40)
        
        # Add arrows from each feature map to its corresponding pooling map
        for row, (ax_output) in output2_block_axes:
            _, ax_pooling = pooling2_block_axes[row]
            line_y_start = ax_output.get_position().y0 + ax_output.get_position().height / 2
            line_y_end = ax_pooling.get_position().y0 + ax_pooling.get_position().height / 2
            line_x_start = ax_output.get_position().x0 + ax_output.get_position().width
            line_x_end = ax_pooling.get_position().x0
            add_straight_arrow(fig, line_x_start, line_y_start, line_x_end, line_y_end, color='purple', width=3.0) 
        
        # Add arrows from each conv3 feature map to its corresponding output
        for row, ax_feature in feature3_block_axes:
            ax_out = output_block_axes[row][1]
            line_y_start = ax_feature.get_position().y0 + ax_feature.get_position().height / 2
            line_y_end = ax_out.get_position().y0 + ax_out.get_position().height / 2
            line_x_start = ax_feature.get_position().x0 + ax_feature.get_position().width
            line_x_end = ax_out.get_position().x0
            add_straight_arrow(fig, line_x_start, line_y_start, line_x_end, line_y_end, color='purple', width=3.0)

        # Plot conv3 bias values on top of the arrow between each feature map and its corresponding output
        for row, ax_feature in feature3_block_axes:
            ax_out = output_block_axes[row][1]
            line_y_start = ax_feature.get_position().y0 + ax_feature.get_position().height / 2
            line_y_end = ax_out.get_position().y0 + ax_out.get_position().height / 2
            line_x_start = ax_feature.get_position().x0 + ax_feature.get_position().width
            line_x_end = ax_out.get_position().x0
            
            # Add the bias value as text on the arrow
            bias_value = self.filter_bias3[row]
            # Plot the bias value as a 1x1 matrix using plot_matrix
            bias_matrix = np.array([[bias_value]])
            # Calculate the position for the bias matrix: center between the two axes
            bias_width = 0.06
            bias_height = 0.05
            bias_center_x = ((line_x_start + line_x_end) / 2) - bias_width / 2
            bias_center_y = ((line_y_start + line_y_end) / 2) - bias_height / 2
            ax_bias = fig.add_axes([bias_center_x, bias_center_y, bias_width, bias_height])
            plot_matrix(ax_bias, bias_matrix, '', show_numbers=True, value_format='.2f', show_grid=False, change_color= False, font = 40)

        # Add an arrow from the sum with the largest value to the prediction box
        if self.predicted_digit is not None:
            max_index = np.argmax(self.out)
            ax_out = output_block_axes[max_index][1]
            line_y_start = ax_out.get_position().y0 + ax_out.get_position().height / 2
            line_y_end = centered_bottom + box_height / 2
            line_x_start = ax_out.get_position().x0 + ax_out.get_position().width
            line_x_end = prediction_left
            add_straight_arrow(fig, line_x_start, line_y_start, line_x_end, line_y_end, color='purple', width=3.0)

        # Add a capital sigma (Σ) symbol above each arrow from the filter 2 channels to filter 2 feature maps
        for block_num, block_output_axes in channels2_block_axes:
            # Get the right center of the 2x2 channel block
            ax_c0 = block_output_axes[0]
            ax_c3 = block_output_axes[3]
            channel_block_x = ax_c3.get_position().x0 + ax_c3.get_position().width + 0.01
            channel_block_y = (ax_c0.get_position().y0 + ax_c3.get_position().y0 + ax_c3.get_position().height) / 2

            # Get the left center of the corresponding feature map in output2_block_axes
            _, ax_feature = output2_block_axes[block_num]
            feature_block_x = ax_feature.get_position().x0 - 0.01
            feature_block_y = ax_feature.get_position().y0 + ax_feature.get_position().height / 2

            # Place the sigma above the arrow, centered between channel block and feature map
            sigma_x = channel_block_x + (feature_block_x - channel_block_x) * 0.25
            sigma_y = channel_block_y + 0.03  # Slightly above the arrow
            fig.text(sigma_x, sigma_y, 'Σ', fontsize=48, verticalalignment='bottom', horizontalalignment='center')
        
        # Add a capital sigma (Σ) symbol above each arrow from the conv3 feature maps to the output
        for row, ax_feature in feature3_block_axes:
            ax_out = output_block_axes[row][1]
            # Calculate the position just above the arrow (slightly above the vertical center)
            line_y = ax_feature.get_position().y0 + ax_feature.get_position().height / 2 + 0.03  # Shift up
            line_x_start = ax_feature.get_position().x0 + ax_feature.get_position().width
            line_x_end = ax_out.get_position().x0

            # Place the sigma above the arrow, not on top of it
            fig.text(line_x_start + 0.02, line_y + 0.03, 'Σ', fontsize=48, verticalalignment='bottom', horizontalalignment='left')

        
        plt.show()

def main():

    try:
        visualizer = CNNVisualizer()
        if visualizer.input_data is not None and np.sum(visualizer.input_data) > 0:
            visualizer.visualize_full_matrices()
        else:
            print("No drawing detected or drawing is empty")
    except FileNotFoundError:
        print("Could not find 'mnist_cnn.pth' model file")
        print("Make sure the CNN is in the same directory")
    except Exception as e:
        print(f"Error: {e}")
        print("Make sure you have the required libraries installed:")
        print("torch torchvision pygame matplotlib scipy")

if __name__ == "__main__":
    main()

- Click and drag to draw
- Press ENTER when finished
- Press SPACE to clear


In [3]:
main()

- Click and drag to draw
- Press ENTER when finished
- Press SPACE to clear
No drawing detected or drawing is empty
